Exercise for Unit 6

BSCS 3A-AI

Luis Rafael L. Ajoc

John Felmer B. Robles

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [2]:
# Load drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "/content/drive/My Drive/Colab Notebooks/Datasets/muffin_vs_chihuahua_dataset/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "/content/drive/My Drive/Colab Notebooks/Datasets/muffin_vs_chihuahua_dataset/test"    # e.g. './muffin-vs-chihuahua/test'

In [4]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [8]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [9]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([

    layers.Conv2D(filters =64, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D(filters =64, kernel_size =3, padding ='same', activation='relu'),
    layers.MaxPool2D(pool_size =2, strides =2, padding ='same'),

    layers.Conv2D(filters =128, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu'),
    layers.MaxPool2D(pool_size =2, strides =2, padding ='same'),

    layers.Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu'),
    layers.MaxPool2D(pool_size =2, strides =2, padding ='same'),

    layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu'),
    layers.MaxPool2D(pool_size =2, strides =2, padding ='same'),

    layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu'),
    layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu'),
    layers.MaxPool2D(pool_size =2, strides =2, padding ='same'),

    layers.Flatten(),
    layers.Dense(units = 4096, activation ='relu'),
    layers.Dense(units = 4096, activation ='relu'),
    layers.Dense(units = 1, activation ='sigmoid')
])

In [10]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 983s 8s/step - accuracy: 0.5233 - loss: 0.7920 - val_accuracy: 0.5407 - val_loss: 0.6903
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 61s 512ms/step - accuracy: 0.5466 - loss: 0.6899 - val_accuracy: 0.5407 - val_loss: 0.6906
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 60s 507ms/step - accuracy: 0.5408 - loss: 0.6908 - val_accuracy: 0.5407 - val_loss: 0.6898
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 61s 515ms/step - accuracy: 0.5484 - loss: 0.6892 - val_accuracy: 0.5407 - val_loss: 0.6899
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 61s 508ms/step - accuracy: 0.5487 - loss: 0.6888 - val_accuracy: 0.5407 - val_loss: 0.6899
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 70s 592ms/step - accuracy: 0.5211 - loss: 0.6932 - val_accuracy: 0.5407 - val_loss: 0.6901
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 61s 511ms/step - accuracy: 0.5546 - loss: 0.6879 - val_accuracy: 0.5407 - val_loss: 0.6899
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 60s 505ms/step - accuracy: 0.5515 - loss: 0.6

In [12]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 420s 12s/step - accuracy: 0.8584 - loss: 0.6369
Test Accuracy: 0.5405405163764954


In [13]:
# SAVE THE MODEL
model.save('/content/drive/My Drive/Colab Notebooks/Models/exercise_6_vgg16.h5', save_format='h5')

In [14]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='/content/drive/My Drive/Colab Notebooks/Models/exercise_6_vgg16.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [15]:
# Example usage:
predict_image("/content/drive/My Drive/Colab Notebooks/Datasets/muffin_vs_chihuahua_dataset/run/run_muffin.jpg")
predict_image("/content/drive/My Drive/Colab Notebooks/Datasets/muffin_vs_chihuahua_dataset/run/run_chihuahua.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Muffin (confidence: 0.46)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step
Prediction: Muffin (confidence: 0.46)
